# Modelado, Simulación y Optimización - Proyecto 1

Amalia Cabonell - xxxxxx \
Santiago Arenas - xxxxxx \
Santiago Casasbuenas - 202214932

## Problema a resolver - Proyecto C

**Objetivo:** El presente proyecto tiene como objetivo diseñar una estrategia integral de rutaspara el transporte de carga a nivel nacional, que permita minimizar los costos ope-rativos y de combustible. Se deberá garantizar la disponibilidad de recarga medianteestaciones de servicio estratégicamente ubicadas, a la vez que se respeta la restricciónde peso impuesta en cada municipio por razones de seguridad vial.


## Parte 1 - Formulación Matemática Completa

### Definición de conjuntos
Se especificará de forma clara los conjuntos relevantes vinculados al problema.

#### 1. Conjunto de Puntos de Acceso (Puertos)

$P$: Conjunto de nodos origen donde se ingresan las mercancias.

Ejemplo: $P = {p_1}$ donde $p_1 = $ Puerto Barranquilla

#### 2. Conjunto de Destinos (Centros de Consumo)

$D$: Conjunto de destinos finales o municipios consumidores.

Ejemplo: $D = {d_1, d_2, d_3}$ donde:

$d_1 = $ Bogotá

$d_2 = $ Guasca

$d_3 = $ Cogua

#### 3. Conjunto de Estaciones de Servicio

$E$: Conjunto de estaciones de recarga de combustibles disponibles en la red.

Ejemplo: $E = {e_1, e_2, e_3}$ donde:

$e_1 = $ Estación La Estrella

$e_2 = $ Estación El Sol

$e_3 = $ Estación Caribe

#### 4. Conjunto de Vehículos

$V$: Conjunto de vehículos disponibles para el transporte.

Ejemplo: $V = {v_1, v_2, v_3}$ donde:

$v_1 = $ Camión de 30 ton

$v_2 = $ Camión de 25 ton

$v_3 = $ Camión de 20 ton

#### 5. Conjunto de "Arcos" Posibles

$A$: Conjunto de arcos entre nodos (puertos, estaciones, destinos) que representan posibles trayectos.

- Cada arco $(i, j)$ indica que es posible moverse del nodo $i$ al nodo $j$.

- Este conjunto se infiere del grafo de la red vial nacional y será determinado con base en la conectividad real y los peajes existentes.

#### 6. Conjunto de Municipios con Restricción de Peso

$M ⊆ D$: Subconjunto de destinos con restricción de peso para los vehículos.

Ejemplo: $E = {d_2, d_3}$ donde Guasca y Cogua tienen límites de peso.

### Definición de Parámetros
Lista de todas las constantes necesarias. Para este escenario, vamos a clasificar los parámetros en grupos de acuerdo con su tema, así serán más faciles de ubicar y diferenciar.

#### 1. Parámetros Geográficos

- $lat_i$: Latitud del nodo i
- $long_i$: Longitud del nodo i

#### 2. Red y Costos de Trayecto

- $dist_{ij}$: Distancia en km entre el nodo $i$ y el nodo $j$, con $(i, j) ∈ A$

- $PeajeBase_{ij}$: Tarifa base de peaje entre nodos $i$ y $j$ (COP).

- $PeajeTon_{ij}$: Costo adicional por tonelada transportada (COP/ton) entre $i$ y $j$
    - Si no aplica, se considera 0.

- $F_t$: Tarifa de flete por km (COP/km) -> 5.000

- $C_m$: Costo de mantenimiento por km (COP/km) -> 700

#### 3. Estaciones de servicio

Para cada estación $e ∈ E$:
- $PrecioCombustible_e$: Precio de combustible en estación $e$ (COP/galón)

#### 4. Vehículos

Para cada vehículo $v ∈ V$:
- $Capacidad_v$: Capacidad útil (ton)
- $Autonomía_v$: Autonomía mpaxima (km)

#### 5. Restricciones de Municipios

Para cada destino $d ∈ D$:
- $PesoMax_d$: Peso máximo permitido en el municipio $d$ (ton)
    - Si no tiene restricción, puede tomarse como un valor muy alto o inf.

#### 6. Parámetros Derivados o Preprocesados
Dependiendo del nivel de detalle del modelo, podríamos pensar en definir parámetros calculados como:
- $CostoTotal_{ijv} = dist_{ij} * (F_t + C_m) + PeajeBase_{ij} + PeajeTon_{ij} * Capacidad_v$

### Variables de Decisión
Detalle de las variables principales y las auxiliares que permiten capturar el comportamiento esperado del sistema. Para facilitar el reconocimiento de variables, las agruparemos en cuatro grupos clave.

#### 1. Variables de Ruta
Variables que nos indican si se utiliza un arco (trayecto) en una ruta para un vehículo determinado.

$$
x_{ij}^v ∈ \text{{0,1}}
$$

$$ 
  x_{ij}^v =
  \begin{cases} 
  1 & \text{si el vehículo } v ∈ V \text{se desplaza del nodo $i$ al nodo $j$} \\ 
  0 & \text{dlc.}
  \end{cases} 
$$

$$
  ∀ (i,j) ∈ A, v ∈ V
$$


#### 2. Variables de Carga Transportada
Variables que permiten modelar el peso que transporta cada vehículo por cada arco:

$$
  w_{ij}^v ≥ 0
$$

- Toneladas de carga transportadas por el vehículo $v$ en el arco $(i,j)$.
- Esto ayuda a validar restricciones de capacidad, peso en municipios, peajes variables por tonelada.

#### 3. Variables de Recarga
Variables que permiten representar la "estrategia" de recarga de combustible.

$$
  r_{i}^v ≥ 0
$$

- Galones de combustible recargados por el vehículo $v$ en la estación $i ∈ E$.
- Si el nodo no es una estación, entonces $r_{i}^v = 0$


#### 4. Variables Auxiliares (para consumo y autonomía)
Control del combustible disponible.

$$
  r_{i}^v ≥ 0
$$

- Cantidad de combustible (galones) disponibles en el tanque del vehículo $v$ al llegar al nodo $i$.

### Restricciones
Restricciones necesarias para representar adecuadamente la realidad del problema. Se agruparán las restricciones para mayor calidad.

#### 1. Restricciones de Flujo
Nos aseguramos de que cada vehículo tenga una ruta continua:

- **Conservación en nodos intermedios (flujo balanceado)**:

   $$
   \sum_{j:(j,i) ∈ A} x_{ij}^v = \sum_{j:(i,j) ∈ A} x_{ij}^v \quad \forall i \in N, v \in V
   $$

- **Salida del punto de acceso**:

   $$
   \sum_{j:(p,j) ∈ A} x_{pj}^v ≤ 1 \quad \forall p \in P, v \in V
   $$


- **Llegada a un destino**:

   $$
   \sum_{j:(i,d) ∈ A} x_{id}^v ≤ 1 \quad \forall d \in D, v \in V
   $$



#### 2. Restricciones de Capacidad y Peso

- **Límite de carga por vehículo**:

   $$
   \sum_{(i,j) ∈ A} w_{ij}^v ≤ Capacidad_v \quad \forall v \in V
   $$


- **Restricción de peso en municipios**:

   $$
   \sum_{i:(i,d) ∈ A} w_{id}^v ≤ PesoMax_d \quad \forall d \in (M ⊆ D), v \in V
   $$

#### 3. Restricciones de Autonomía y Consumo

- **Estado de combustible al llegar al nodo j**:

   $$
   f_{j}^v = f_{i}^v - Consumo_v * dist_{ij} + f_{j}^v \quad \forall (i,j) \in A
   $$

- **Autonomía máxima del taque**:

   $$
   f_{i}^v = Autonomía_v \quad \forall i \in N, v \in V
   $$

- **No recarga si el nodo no estación**:

   $$
   r_{i}^v = 0 \quad \forall i \notin E
   $$

#### 4. Cosotos de Peaje con peso
OJO: Esto se puede incorporar directamente en la función objetivo como término de costo

- **Si hay costo por tonelada**: 
   $$
   Peaje_{ij}^v = PeajeBase_{ij} + PeajeTon_{ij} * w_{ij}^v
   $$

#### 5. Entrega Total de Carga
(Si se conoce la demanda total a transportar. Toca preguntar pq no entendí esa parte o no leí bien, la dejo por si acaso).

   $$
   \sum_{v ∈ V} \sum_{i:(i,d) ∈ A} w_{id}^v = Demanda_d \quad \forall d \in D
   $$

#### 6. Consistencia entre variables

- **Solo puede haber carga o recarga si el arco (ruta) es usado**:

   $$
   w_{ij}^v ≤ Capacidad_{v} * x_{ij}^v , r_{i}^v ≤ R_{max} * \sum_{j} x_{ij}^v
   $$

## Parte 2 - Función Objetivo y Proceso de indagación

### Función objetivo 
Planteamiento explícito de la función a optimizar considerando posibles penalizaciones por incumplimiento de restricciones.

### Descripción del proceso de indagación
Con el fin de definir y justificar la función objetivo, incluyendo la revisión de literatura y análisis comparativo con casos de estudio relevantes.

## Parte 3 - Preprocesamiento de Datos y Análisis

### Procesos utilizados para extraer y procesar datos adicionales

### Integración de la información en el modelo

## Parte 4 - Ejemplos Ilustrativos

### Casos de aplicación del modelo

### Análisis de factibilidad y consistencia de la solución

## Parte 5 - Justificación y Discusión

### Justificación de los elementos del modelo

### Discusión sobre relevancia de los datos, metodologías adoptadas y posibles impactos operativos